Подключаем ВАШ гугл-диск, как папку. В примере у меня в корне моего диска есть папка weather_data, где лежит файл boloto_2016.xlsx

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from google.colab import drive
import os
drive.mount('/content/drive')

import pandas as pd
import numpy as np

In [ ]:
target = 'NEE'#'co2_flux_full output'
fileName = 'weather_data/for_filtering.xlsx'
z_val = 5.5

In [ ]:
all_data = pd.read_excel(os.path.join("/content/drive/MyDrive/", fileName),
                         sheet_name=None,          # загружаем все листы! Если надо конкретные, можно их прописать тут
                         skiprows=None)#lambda x: x==1)  # Важный момент! В моем файле вторая строка идет без данных, поэтому я ее вот таким образом пропускаю, уберите эту опцию, если у вас все идет подряд.
data = pd.concat(all_data.values(), ignore_index=True)

In [ ]:
data['dateTime'] = pd.to_datetime(data['timestamp'],
                                  format='%Y-%m-%d %H:%M:%S') #это шаблон того, как у нас выглядят наши данные.
data.index = pd.DatetimeIndex(data['dateTime'])

#data['dateTime'] = data['dateTime'].astype(str)
# data = data.rename(columns={'Unnamed: 0': "dateTime"})
# data['dateTime'] = pd.to_datetime(data['dateTime'],
#                                   format='%d.%m.%Y %H:%M:%S') #это шаблон того, как у нас выглядят наши данные.
# data.index = pd.DatetimeIndex(data['dateTime'])



# data = data.rename(columns={'Unnamed: 0': "dateTime"})


In [ ]:
data['process_flag'] = np.invert(pd.isnull(data[target]))

Это наши функции, которые мы будем применять.

In [ ]:
def hampel_filter_pandas(input_series, window_size, n_sigmas=3):
    k = 1.4826  # scale factor for Gaussian distribution
    new_series = input_series.copy()
    indices = None
    # # helper lambda function
    MAD = lambda x: np.median(np.abs(x - np.median(x)))

    rolling_median = input_series.rolling(window=2 * window_size, center=True).median()
    rolling_mad = k * input_series.rolling(window=2 * window_size, center=True).apply(MAD)
    diff = np.abs(input_series - rolling_median)
    print(len(diff.index), len(rolling_mad.index))

    indices = diff > (n_sigmas * rolling_mad)  # list(np.argwhere(diff > (n_sigmas * rolling_mad)).flatten())
    # #new_series[indices] = rolling_median[indices]

    return new_series, indices[indices == True].index

def mad_filter(input_df, target_col, z=5.5, fill_method='new'):
    data_if = input_df[[target_col]].copy()
    data_if['rolling_fill'] = data_if[target_col].rolling(10).mean()
    data_if['plus_shift'] = data_if[target_col].shift(1)
    null_index_plus = data_if['plus_shift'].isnull()
    null_index_plus = null_index_plus[null_index_plus == True].index

    data_if['minus_shift'] = data_if[target_col].shift(-1)
    null_index_minus = data_if['minus_shift'].isnull()
    null_index_minus = null_index_minus[null_index_minus == True].index

    if fill_method == 'old':
        data_if.loc[null_index_plus, 'plus_shift'] = data_if.loc[null_index_plus, target_col]
        data_if.loc[null_index_minus, 'minus_shift'] = data_if.loc[null_index_minus, target_col]
    else:
        data_if.loc[null_index_plus, 'plus_shift'] = data_if.loc[null_index_plus, 'rolling_fill']
        data_if.loc[null_index_minus, 'minus_shift'] = data_if.loc[null_index_minus, 'rolling_fill']

    data_if['d_i'] = (data_if[target_col] - data_if['minus_shift']) - (data_if['plus_shift'] - data_if[target_col])

    # print(key, pd.isna(data['d_i']).value_counts())
    d_median = np.median(data_if.query('not (d_i != d_i)')['d_i'])
    MAD = np.median(np.abs(data_if.query('not (d_i != d_i)')['d_i'] - d_median))

    down_threshold = d_median - (z * MAD / 0.6745)
    up_threshold = d_median + (z * MAD / 0.6745)

    out_data = np.logical_or(data_if['d_i'] < down_threshold, data_if['d_i'] > up_threshold, data_if['d_i'].isna())
    return out_data

In [ ]:
data.loc[data['process_flag']==True, 'filter_55'] = mad_filter(data.query('process_flag==True'), target, z=5.5, fill_method='new')
data['filter_hampel'] = False
rwn, indices = hampel_filter_pandas(data.query('process_flag==True')[target], window_size=10)
data.loc[indices, 'filter_hampel'] = True

# fig = px.scatter(data, x='dateTime', y=target, color='filter_55', title=f"co2 MAD z={z_val}")
# # fig.write_html(f"/content/drive/MyDrive/weather_data/co2_plot.html")
# fig.update_yaxes(type="log")
# fig.show()


data['Outlier'] = False
rwn, indices = hampel_filter_pandas(data.query('process_flag==True & filter_55==False')[target], window_size=10)
data.loc[indices, 'Outlier'] = True
data['Outlier'] = np.logical_or(data['Outlier'], data['filter_55'])
fig = px.scatter(data.query('process_flag==True'), x='dateTime', y=target, color='Outlier', title=f'co2 Hampel after MAD {z_val}')
fig.show()

In [ ]:
base_dir, out_filename =  os.path.split(os.path.join("/content/drive/MyDrive/", fileName))
out_filename = "out_"+out_filename
data.to_csv(os.path.join(base_dir, out_filename))
